This is a notebook that can be used to pull data from the iRacing API.

https://forums.iracing.com/discussion/15068/general-availability-of-data-api/p1

uses the [iracingdataapi package](https://github.com/jasondilworth56/iracingdataapi) to pull data from the iRacing API.



In [ ]:
!pip install git+https://github.com/jasondilworth56/iracingdataapi.git
!pip install gspread oauth2client pandas

In [21]:
import time
import csv
import json
from iracingdataapi.client import irDataClient
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

import os

# this is the event type
Race = 5

# Environment variables are used for sensitive information
# You can set these in your shell or in your IDE
# alternatively you can set them directly in the code below by adjusting the values of the variables
username = os.getenv("IRACING_USERNAME")
password = os.getenv("IRACING_PASSWORD")
svc_acct_key_file = os.getenv("F499_SVC_ACCT_KEY_FILE")


# alternatively you can set the username and password directly here
# fill in your username and password, an uncomment the lines below
# username = "putyourusernamehereitisusuallyyouremailaddress"
# password = "putyourpasswordhere"

def get_gspread_client():
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']

    creds = ServiceAccountCredentials.from_json_keyfile_name(svc_acct_key_file, scope)
    client = gspread.authorize(creds)
    return client


def get_gspread_sheet(client, sheet_name, worksheet_id=None):
    if worksheet_id is not None:
        return client.open(sheet_name).get_worksheet_by_id(worksheet_id)
    else:
        return client.open(sheet_name).sheet1


def upload_to_gspread(sheet_name, data_csv_filename):
    client = get_gspread_client()
    sheet = get_gspread_sheet(client, sheet_name)
    df = pd.read_csv(data_csv_filename)
    sheet.update([df.columns.values.tolist()] + df.values.tolist())


def calculate_f499_points(start, finish, incident_count):
    return (start - finish) + (4 if incident_count == 0 else -1 * (2 * incident_count))


def get_session_link(subsession_id, new_ui=False):
    if new_ui:
        format = "https://members-ng.iracing.com/racing/results-stats/results?subsessionid="
    else:
        format = "https://members.iracing.com/membersite/member/EventResult.do?&subsessionid="

    return f"{format}{subsession_id}"


def write_results_to_file(results, filename):
    # Write the raw results to a JSON file
    with open(f"{filename}_raw_results.json", 'w') as jsonfile:
        json.dump(results, jsonfile)


def write_results_to_csv_file(race_data_list, filename):
    # Specify the order of your columns
    fieldnames = ["season_year",
                  "season_quarter",
                  "week_number",
                  "racer_name",
                  "cust_id",
                  "series_name",
                  "series_id",
                  "start_time",
                  "track_name",
                  "session_link",
                  "subsession_id",
                  "start_position",
                  "finish_position",
                  "incident_count",
                  "_499_points"
                  ]

    # Open your CSV file in write mode
    with open(f'{filename}_data.csv', 'w', newline='') as csvfile:
        # Create a CSV writer object
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the header to the CSV file
        writer.writeheader()

        # Write each dictionary in the list to the CSV file
        for race_data_dict in race_data_list:
            writer.writerow(race_data_dict)


def print_race_results(race_data):
    print(f"Race {race_data['start_time']}:")
    print(f"Session Link: [{race_data['subsession_id']}]({race_data['session_link']})")
    print(f"Series: {race_data['series_name']}")
    print(f"Track: {race_data['track_name']}")
    print(f"Start Position: {race_data['start_position'] if race_data['start_position'] else 'N/A'}")
    print(f"Finish Position: {race_data['finish_position'] if race_data['finish_position'] else 'N/A'}")
    print(f"Number of Incidents: {race_data['incident_count']}")
    print(f"F499 Scoring Points: {race_data['_499_points']}")
    print()


def get_499_series(client: irDataClient):
    series = client.series
    # find the series with the string "LMP3" in the name
    _499_series = [s for s in series if "LMP3" in s['series_name']]
    # # add any series that has the string "LMP2" in the name to the list
    _499_series.extend([s for s in series if "LMP2" in s['series_name']])
    # # add any series that has the string "F4" in the name to the list
    _499_series.extend([s for s in series if "FIA Formula 4" in s['series_name']])
    # # add any series that has the string "Falken Sports Car" in the name to the list
    _499_series.extend([s for s in series if "Falken Tyre Sports Car" in s['series_name']])
    # add any series that has the string "IMSA" in the name to the list
    _499_series.extend([s for s in series if "IMSA iRacing Series" in s['series_name']])

    # put the series_id and series_name in a tuple and add them to a list
    _499_series = [(s['series_id'], s['series_name']) for s in _499_series]
    return _499_series


def construct_499_race_data(raw_result, racer_name):
    start_position = raw_result['starting_position_in_class'] + 1
    finish_position = raw_result['finish_position_in_class'] + 1
    week_number = raw_result['race_week_num'] + 1
    _499_points = calculate_f499_points(start_position, finish_position, raw_result['incidents'])
    session_link = get_session_link(raw_result['subsession_id'], True)

    return {
        "start_position": start_position,
        "finish_position": finish_position,
        "incident_count": (raw_result['incidents']),
        "track_name": (raw_result['track']['track_name']),
        "subsession_id": (raw_result['subsession_id']),
        "start_time": (raw_result['start_time']),
        "week_number": week_number,
        "season_year": (raw_result['season_year']),
        "season_quarter": (raw_result['season_quarter']),
        "_499_points": _499_points,
        "session_link": session_link,
        "series_name": (raw_result['series_name']),
        "series_id": (raw_result['series_id']),
        "racer_name": racer_name,
        "cust_id": raw_result['cust_id'],
    }


def append_to_gspread(sheet_name, worksheet_id, data_csv_filename, client):
    sheet = get_gspread_sheet(client, sheet_name, worksheet_id)

    # Read existing data from the Google Sheet, skipping the first two rows
    existing_df = pd.DataFrame(sheet.get_all_records())

    # # Load new data from the CSV file  skipping the first row
    new_df = pd.read_csv(data_csv_filename)

    # Append new data to the existing data using pd.concat
    df = pd.concat([existing_df, new_df], ignore_index=True)

    # drop duplicates based on subsession_id and cust_id
    df.drop_duplicates(subset=['subsession_id', 'cust_id'], inplace=True)

    # Clear the sheet so that we can write the now complete new data
    sheet.clear()
    # write df to the sheet, including the header
    sheet.update([df.columns.values.tolist()] + df.values.tolist())

    # sor the df by start_time, descending and cust_id
    df.sort_values(by=['start_time', 'cust_id'], ascending=[False, True], inplace=True)

    # batch_format appropriate columns to be numbers with no decimal places
    sheet.batch_format([{'range': 'A:A', 'format': {'numberFormat': {'type': 'NUMBER', 'pattern': '0'}}},
                        {'range': 'B:B', 'format': {'numberFormat': {'type': 'NUMBER', 'pattern': '0'}}},
                        {'range': 'C:C', 'format': {'numberFormat': {'type': 'NUMBER', 'pattern': '0'}}},
                        {'range': 'E:E', 'format': {'numberFormat': {'type': 'NUMBER', 'pattern': '0'}}},
                        {'range': 'G:G', 'format': {'numberFormat': {'type': 'NUMBER', 'pattern': '0'}}},
                        {'range': 'K:K', 'format': {'numberFormat': {'type': 'NUMBER', 'pattern': '0'}}},
                        {'range': 'L:L', 'format': {'numberFormat': {'type': 'NUMBER', 'pattern': '0'}}},
                        {'range': 'M:M', 'format': {'numberFormat': {'type': 'NUMBER', 'pattern': '0'}}},
                        {'range': 'N:N', 'format': {'numberFormat': {'type': 'NUMBER', 'pattern': '0'}}},
                        {'range': 'O:O', 'format': {'numberFormat': {'type': 'NUMBER', 'pattern': '0'}}}
                        ])


def get_participants_from_sheet(google_client, sheet_name, worksheet_id):
    retrieve_participants = []
    # get participants from the sheet named "F499 Tracker" and worksheet id 1273007515
    sheet = get_gspread_sheet(google_client, sheet_name, worksheet_id)
    df = pd.DataFrame(sheet.get_all_records())
    for index, row in df.iterrows():
        retrieve_participants.append((row['cust_id'], row['driver_name']))

    return retrieve_participants

def get_series_from_sheet(google_client, sheet_name, worksheet_id):
    retrieve_series = []
    # get series from the sheet named "F499 Tracker" and worksheet id 2085237774
    sheet = get_gspread_sheet(google_client, sheet_name, worksheet_id)
    df = pd.DataFrame(sheet.get_all_records())
    for index, row in df.iterrows():
        retrieve_series.append((row['series_id'], row['series_name']))

    return retrieve_series


def run():
    idc = irDataClient(username=username, password=password)
    google_client = get_gspread_client()
    tracker_sheet_name = "F499 Tracker"
    results_worksheet_id = 1416763316
    participant_worksheet_id = 935466926
    participant_form_worksheet_id = 1422405375
    series_worksheet_id = 2085237774

    desired_season_year = 2024
    desired_season_quarter = 3
    desired_season_week = 3

    raw_results = []
    race_data_list = []
    
    participants = get_participants_from_sheet(google_client, tracker_sheet_name, participant_worksheet_id)
    series_of_interest = get_series_from_sheet(google_client, tracker_sheet_name, series_worksheet_id)

    # iterate over the participants
    for cust_id, driver_name in participants:
        title = f"{desired_season_year}S{desired_season_quarter} Week {desired_season_week} CustID {cust_id}"
        print(f"{driver_name} - Customer ID: {cust_id}, {title}")

        for series_id, series_name in series_of_interest:
            # pause for a half second to avoid hitting the API rate limit
            time.sleep(0.5)
            # NOTE: race_week_num seems to do nothing...
            series_results = idc.result_search_series(cust_id=cust_id,
                                                      series_id=series_id,
                                                      official_only=True,
                                                      event_types=[Race],
                                                      season_year=desired_season_year,
                                                      season_quarter=desired_season_quarter,
                                                      race_week_num=desired_season_week - 1)
            print(f"Series: {series_name}")
            print(f"{len(series_results)} Races")

            for i, result in enumerate(series_results, start=1):
                raw_results.append(result)
                race_data = construct_499_race_data(result, driver_name)
                race_data_list.append(race_data)

        # sort race_data_list by start_time, descending
        race_data_list.sort(key=lambda x: x['start_time'])

        base_filename = title.replace(" ", "_")
        write_results_to_csv_file(race_data_list, base_filename)
        write_results_to_file(raw_results, base_filename)

        # add a 1 second delay to avoid hitting the API rate limit
        time.sleep(1)
        append_to_gspread(tracker_sheet_name, results_worksheet_id, f"{base_filename}_data.csv", google_client)

        race_data_list = []
        print(f"Uploaded {base_filename}_data.csv to Google Sheets")
    print("Done")


run()


Matt Barrett - Customer ID: 898530, 2024S3 Week 3 CustID 898530
Series: LMP3 Trophy - Fixed
3 Races
Series: LMP2 Prototype  Challenge – Fixed
0 Races
Series: FIA Formula 4 Challenge
0 Races
Series: FIA Formula 4 Challenge - Fixed
0 Races
Series: Falken Tyre Sports Car Challenge
4 Races
Series: IMSA iRacing Series - Fixed
0 Races
Series: IMSA iRacing Series
0 Races
Uploaded 2024S3_Week_3_CustID_898530_data.csv to Google Sheets
Remko Barlow - Customer ID: 601298, 2024S3 Week 3 CustID 601298
Series: LMP3 Trophy - Fixed
0 Races
Series: LMP2 Prototype  Challenge – Fixed
0 Races
Series: FIA Formula 4 Challenge
0 Races
Series: FIA Formula 4 Challenge - Fixed
0 Races
Series: Falken Tyre Sports Car Challenge
0 Races
Series: IMSA iRacing Series - Fixed
0 Races
Series: IMSA iRacing Series
0 Races
Uploaded 2024S3_Week_3_CustID_601298_data.csv to Google Sheets
YouichiHophop - Customer ID: 95169, 2024S3 Week 3 CustID 95169
Series: LMP3 Trophy - Fixed
0 Races
Series: LMP2 Prototype  Challenge – Fixed